In [2]:
from datasets import load_dataset

dataset = load_dataset("databricks/databricks-dolly-15k")

Generating train split: 15011 examples [00:00, 190879.65 examples/s]


In [8]:
from dataclasses import dataclass
from typing import Optional
import torch

@dataclass
class Question:
    question: str
    context_index: int
    embedding: Optional[torch.Tensor] = None
    transformed_embedding: Optional[torch.Tensor] = None

@dataclass
class Context:
    context: str
    context_index: int
    embedding: Optional[torch.Tensor] = None
    transformed_embedding: Optional[torch.Tensor] = None

@dataclass
class DataCollection:
    questions: list[Question]
    contexts: list[Context]
    metadata: dict


In [12]:
df = dataset['train'].to_pandas()

In [13]:
df = df[df['category'] == "closed_qa"]

In [14]:
df

,instruction,context,response,category
0,When did Virgin Australia start operating?,"Virgin Australia, the trading name of Virgin A...",Virgin Australia commenced services on 31 Augu...,closed_qa
4,When was Tomoaki Komorida born?,Komorida was born in Kumamoto Prefecture on Ju...,"Tomoaki Komorida was born on July 10,1981.",closed_qa
6,"Given a reference text about Lollapalooza, whe...",Lollapalooza /ˌlɒləpəˈluːzə/ (Lolla) is an ann...,Lollapalooze is an annual musical festival hel...,closed_qa
11,Who was Kyle Van Zyl playing against when he s...,Van Zyl joined the Eastern Province Kings Acad...,Kyle Van Zyl was playing against Boland U21 wh...,closed_qa
48,Who was the first to record the Needles and Pi...,"""Needles and Pins"" is a rock song credited to ...",The first artist to record Needles and Pins so...,closed_qa
...,...,...,...,...
14988,How many types of cat does the world cat feder...,"As of 2023, The International Cat Association ...",69,closed_qa
14990,What is the history of coffee?,The history of coffee dates back to centuries ...,Much of the early history of coffee was tied t...,closed_qa
15001,What are common florals found in Zigalga Natio...,Zigalga National Park (Russian: Национальный п...,Zigalga National Park has the majority of its ...,closed_qa
15003,What is linux Bootloader,"A bootloader, also spelled as boot loader or c...",A bootloader is a program written in machine c...,closed_qa


In [15]:
questions = []
contexts = []
df = df.reset_index()
for index, row in df.iterrows():
    questions.append(
        Question(row['instruction'], index, None, None)
    )
    contexts.append(Context(row['context'], index, None, None))

In [16]:
collection = DataCollection(questions, contexts, metadata={
    "category":"dolly"
})

In [18]:
import pickle
with open("data/raw/dolly-base.pkl", 'wb') as file:
    pickle.dump(collection, file)